In [1]:
import sys
sys.path.insert(0, '..')

import logging

import codecs
import numpy as np
import pandas as pd

from app.config import (
    configure_logging,
    CURRENT_YEAR
)
from app import db

OUTPUT_TABLE_NAME = 'play_by_play_enriched'

In [2]:
db_conn = db.get_db_conn()

In [4]:
def _convert_to_gsis_id(pbp_id):
    """Convert the play by play id to gsis id"""
    if pbp_id is None:
        return None

    if type(pbp_id) == float:
        return pbp_id

    return codecs.decode(pbp_id[4:-8].replace('-', ''), "hex").decode('utf-8')


def _extract(db_conn) -> pd.DataFrame:
    """Get the upstream tables."""
    logging.info("Extracting play by play data...")
    query = """SELECT * FROM play_by_play"""
    df_play_by_play = pd.read_sql(query, db_conn)
    logging.info(f"Extracted {len(df_play_by_play)} rows of play by play data.")

    logging.info("Extracting roster data...")
    query = """SELECT * FROM roster"""
    df_roster = pd.read_sql(query, db_conn)
    logging.info(f"Extracted {len(df_roster)} rows of roster data.")

    return df_play_by_play, df_roster

In [5]:
df_play_by_play, df_roster = _extract(db_conn)

In [6]:
df_play_by_play['passer'] = np.where(
    df_play_by_play['passer'].isna() & ~df_play_by_play['passer_player_name'].isna(),
    df_play_by_play['passer_player_name'],
    df_play_by_play['passer']
)

In [7]:
df_play_by_play['passer_gsis_id'] = df_play_by_play['passer_id'].apply(_convert_to_gsis_id)
df_play_by_play['receiver_gsis_id'] = df_play_by_play['receiver_id'].apply(_convert_to_gsis_id)
df_play_by_play['rusher_gsis_id'] = df_play_by_play['rusher_id'].apply(_convert_to_gsis_id)

In [8]:
logging.info("Adding position data from roster table...")
df_roster_slim = df_roster[['season', 'position', 'gsis_id']]
df_roster_slim = df_roster_slim.drop_duplicates()
df_roster_slim = df_roster_slim[df_roster_slim['position'].notnull()]
df_roster_slim = df_roster_slim[df_roster_slim['gsis_id'].notnull()]

In [22]:
df_enriched = df_play_by_play.merge(
    df_roster_slim,
    how='left',
    left_on=['season', 'receiver_gsis_id'],
    right_on=['season', 'gsis_id']
)
df_enriched = df_enriched.rename(columns={'position': 'receiver_position'})
df_enriched = df_enriched.drop('gsis_id', axis=1)
df_enriched['receiver_position'].value_counts()

WR    11527
TE     3989
RB     3445
FB      138
QB       34
T         5
LB        4
C         2
CB        2
K         1
DT        1
Name: receiver_position, dtype: int64

In [23]:
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,0.060081,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,0.342128,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,0.524574,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN


In [24]:
df_enriched = df_enriched.merge(
    df_roster_slim,
    how='left',
    left_on=['season', 'rusher_gsis_id'],
    right_on=['season', 'gsis_id']
)
df_enriched = df_enriched.drop('gsis_id', axis=1)
df_enriched = df_enriched.rename(columns={'position': 'rusher_position'})
df_enriched['rusher_position'].value_counts()

RB    11852
QB     1399
WR      471
FB       54
TE       24
DB        7
P         5
SS        5
LB        2
C         1
CB        1
Name: rusher_position, dtype: int64

In [25]:
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN


In [26]:
df_enriched = df_enriched.merge(
    df_roster_slim,
    how='left',
    left_on=['season', 'passer_gsis_id'],
    right_on=['season', 'gsis_id']
)
df_enriched = df_enriched.rename(columns={'position': 'passer_position'})
df_enriched = df_enriched.drop('gsis_id', axis=1)
df_enriched['passer_position'].value_counts()

QB    21973
WR       45
P        14
RB        5
TE        3
DB        1
Name: passer_position, dtype: int64

In [27]:
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position,passer_position
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN


In [29]:
df_roster['player'] = df_roster.apply(lambda row: f"{row['first_name'][0]}.{row['last_name']}", axis=1)
df_roster_slim = df_roster[['season', 'team', 'player', 'position', 'gsis_id']]
df_roster_slim = df_roster_slim.drop_duplicates()
df_roster_slim = df_roster_slim[df_roster_slim['position'].notnull()]
df_roster_slim = df_roster_slim[df_roster_slim['gsis_id'].notnull()]
df_roster_slim = df_roster_slim.rename(
    columns={
        'gsis_id': "gsis_id_2",
        'position': 'position_2'
    }
)
df_roster_slim.head()

,season,team,player,position_2,gsis_id_2
0,2020,ARI,M.Cole,C,00-0034785
1,2020,ARI,L.Gaillard,C,00-0035536
2,2020,ARI,B.Murphy,CB,00-0035236
3,2020,ARI,P.Amukamara,CB,00-0027957
4,2020,ARI,J.Whittaker,CB,00-0036043


In [30]:
df_enriched = df_enriched.merge(
    df_roster_slim,
    how='left',
    left_on=['season', 'posteam', 'passer'],
    right_on=['season', 'team', 'player']
)
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position,passer_position,team,player,position_2,gsis_id_2
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,None,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,None,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,None,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869


In [31]:
df_enriched['passer_gsis_id'] = np.where(
    df_enriched['passer_gsis_id'].isna(),
    df_enriched['gsis_id_2'],
    df_enriched['passer_gsis_id']
)
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position,passer_position,team,player,position_2,gsis_id_2
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,NaN,NYJ,S.Darnold,QB,00-0034869


In [32]:
df_enriched['passer_position'] = np.where(
    df_enriched['passer_position'].isna(),
    df_enriched['position_2'],
    df_enriched['passer_position']
)
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position,passer_position,team,player,position_2,gsis_id_2
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,QB,NYJ,S.Darnold,QB,00-0034869
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,QB,NYJ,S.Darnold,QB,00-0034869
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,00-0034869,None,None,NaN,NaN,QB,NYJ,S.Darnold,QB,00-0034869


In [33]:
df_enriched = df_enriched.drop(['team', 'player','gsis_id_2', 'position_2'], axis=1)
df_enriched[
    (df_enriched['passer'] == 'S.Darnold')
    & (df_enriched['play_type'] == 'qb_spike')
]

,year,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,passer_gsis_id,receiver_gsis_id,rusher_gsis_id,receiver_position,rusher_position,passer_position
1898,2020,2223.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,00-0034869,None,None,NaN,NaN,QB
1903,2020,2358.0,2020_01_NYJ_BUF,2.020091e+09,BUF,NYJ,REG,1.0,NYJ,away,...,NaN,NaN,NaN,NaN,00-0034869,None,None,NaN,NaN,QB
20748,2020,1939.0,2020_08_NYJ_KC,2.020110e+09,KC,NYJ,REG,8.0,NYJ,away,...,NaN,NaN,NaN,NaN,00-0034869,None,None,NaN,NaN,QB
